# Product-mix Scipy

$$
\begin{align}
    \text{max} \quad & \sum_{j \in J} c_j x_j \\
    \text{s.t.} \quad & \sum_{j \in J} a_{i, j} x_{j} \leq b_{i} & \forall \; i \in I \\
    & x_{j} \geq 0 & \forall \; j \in J \\
\end{align}
$$

In [1]:
# Python native modules
import json

# Third-party packages
import numpy as np
from scipy.optimize import linprog

In [2]:
# Read input file and store in local variable `input_data`
with open("input_prod_mix.json", mode="r", encoding="utf8") as file:
    input_data = json.load(file)

In [3]:
input_data

{'margins': [{'product': 'P1', 'value': 2.15},
  {'product': 'P2', 'value': 1.34},
  {'product': 'P3', 'value': 1.72}],
 'availabilities': [{'resource': 'R1', 'value': 80},
  {'resource': 'R2', 'value': 30},
  {'resource': 'R3', 'value': 25}],
 'proportions': [{'resource': 'R1', 'product': 'P1', 'proportion': 0.7},
  {'resource': 'R1', 'product': 'P2', 'proportion': 0.3333333333333333},
  {'resource': 'R1', 'product': 'P3', 'proportion': 0.5},
  {'resource': 'R2', 'product': 'P1', 'proportion': 0.2},
  {'resource': 'R2', 'product': 'P2', 'proportion': 0.6666666666666666},
  {'resource': 'R2', 'product': 'P3', 'proportion': 0.16666666666666666},
  {'resource': 'R3', 'product': 'P1', 'proportion': 0.1},
  {'resource': 'R3', 'product': 'P2', 'proportion': 0.0},
  {'resource': 'R3', 'product': 'P3', 'proportion': 0.3333333333333333}]}

In [4]:
# Sets (with corresponding indexes)
resources = {item["resource"]: j for j, item in enumerate(input_data["availabilities"])}
products = {item["product"]: i for i, item in enumerate(input_data["margins"])}

# Parameters
availabilities = {item["resource"]: item["value"] for item in input_data["availabilities"]}
margins = {item["product"]: item["value"] for item in input_data["margins"]}
proportions = {(item["resource"], item["product"]): item["proportion"] for item in input_data["proportions"]}

In [8]:
resources

{'R1': 0, 'R2': 1, 'R3': 2}

In [9]:
products

{'P1': 0, 'P2': 1, 'P3': 2}

In [10]:
availabilities

{'R1': 80, 'R2': 30, 'R3': 25}

In [11]:
margins

{'P1': 2.15, 'P2': 1.34, 'P3': 1.72}

In [12]:
proportions

{('R1', 'P1'): 0.7,
 ('R1', 'P2'): 0.3333333333333333,
 ('R1', 'P3'): 0.5,
 ('R2', 'P1'): 0.2,
 ('R2', 'P2'): 0.6666666666666666,
 ('R2', 'P3'): 0.16666666666666666,
 ('R3', 'P1'): 0.1,
 ('R3', 'P2'): 0.0,
 ('R3', 'P3'): 0.3333333333333333}

In [5]:
# Create fixed parameters
c = np.empty(len(products))
b = np.empty(len(resources))
A = np.empty((len(resources), len(products)))


# Fill parameters
for r, i in resources.items():
    b[i] = availabilities[r]

for p, j in products.items():
    c[j] = -margins[p]

for r, i in resources.items():
    for p, j in products.items():
        A[i, j] = proportions[r, p]

In [6]:
# Apply linprog function
sol = linprog(c, A_ub=A, b_ub=b, bounds=(0, None))

In [13]:
sol

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -258.8526315789474
              x: [ 7.105e+01  1.026e+01  5.368e+01]
            nit: 3
          lower:  residual: [ 7.105e+01  1.026e+01  5.368e+01]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00]
          upper:  residual: [       inf        inf        inf]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00]
          eqlin:  residual: []
                 marginals: []
        ineqlin:  residual: [ 0.000e+00  0.000e+00  0.000e+00]
                 marginals: [-2.805e+00 -6.074e-01 -6.484e-01]
 mip_node_count: 0
 mip_dual_bound: 0.0
        mip_gap: 0.0

In [7]:
# Print results
for j, xi in enumerate(sol.x):
    print(f"{xi:.2f} units of Product {j}")

71.05 units of Product 0
10.26 units of Product 1
53.68 units of Product 2
